In [1]:
import numpy as np
import open3d as o3d
import json
from collections import defaultdict

Tooth_order = [48, 47, 46, 45, 44, 43, 42, 41, 31, 32, 33, 34, 35, 36, 37, 38]

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
def read_ply_file(ply_path):
    """
    Read PLY file and preserve vertex ordering
    """
    mesh = o3d.io.read_triangle_mesh(ply_path)
    vertices = np.asarray(mesh.vertices)
    faces = np.asarray(mesh.triangles) + 1  # Add 1 to match OBJ 1-based indexing
    
    if mesh.has_vertex_colors():
        rgb = np.asarray(mesh.vertex_colors)
    else:
        rgb = np.full((len(vertices), 3), 0.501)
    
    print(f"Read PLY file: {len(vertices)} vertices, {len(faces)} faces")
    return vertices, rgb, faces

def validate_fdi(fdi):
    """Validate if FDI number is in expected tooth order"""
    try:
        fdi_num = int(float(fdi))  # Handle both string and float FDI numbers
        return fdi_num in Tooth_order
    except (ValueError, TypeError):
        return False

def process_jaw_ply_files(lower_ply_path, upper_ply_path, labels_json_path=None):
    """
    Process jaw PLY files and maintain spatial relationships
    """
    # Convert PLY files to OBJ format
    lower_obj_path = lower_ply_path.replace('.ply', '.obj')
    upper_obj_path = upper_ply_path.replace('.ply', '.obj')
    
    # Get vertex count while converting
    lower_obj_path, vertex_count = convert_ply_to_obj(lower_ply_path, lower_obj_path)
    convert_ply_to_obj(upper_ply_path, upper_obj_path)
    
    if labels_json_path:
        try:
            with open(labels_json_path, 'r') as f:
                labels_data = json.load(f)
            
            # Initialize labels array
            fdi_labels = np.zeros(vertex_count, dtype=int)
            
            # Handle the JSON format
            if isinstance(labels_data, dict):
                if "labels" in labels_data:
                    # Direct label format
                    input_labels = labels_data["labels"][:vertex_count]
                    for i, label in enumerate(input_labels):
                        if validate_fdi(label):
                            fdi_labels[i] = int(float(label))
                elif "cells" in labels_data:
                    # Cell-based format
                    cells = labels_data["cells"]
                    for cell in cells:
                        fdi = cell.get("fdi")
                        if not validate_fdi(fdi):
                            continue
                        vertices = cell.get("vertices", [])
                        for vertex_idx in vertices:
                            if vertex_idx < vertex_count:
                                fdi_labels[vertex_idx] = int(float(fdi))
            
            # Create output in the same format
            converted_labels = {
                "jaw": "lower",
                "number_of_teeth": vertex_count,
                "labels": fdi_labels.tolist()
            }
            
            # Save processed labels
            output_json_path = lower_obj_path.replace('.obj', '.json')
            with open(output_json_path, 'w') as f:
                json.dump(converted_labels, f)
            
            # Print statistics
            valid_fdis = sorted(set(fdi for fdi in fdi_labels if fdi != 0))
            print(f"\nProcessed {len(fdi_labels)} labels for {vertex_count} vertices")
            print(f"Valid FDI numbers found: {valid_fdis}")
            
            # Print vertex distribution
            print("\nVertex distribution per tooth:")
            total_labeled = 0
            for fdi in Tooth_order:
                count = np.sum(fdi_labels == fdi)
                if count > 0:
                    print(f"Tooth {fdi}: {count} vertices")
                    total_labeled += count
            
            print(f"\nTotal labeled vertices: {total_labeled}")
            print(f"Unlabeled vertices: {vertex_count - total_labeled}")
            
        except Exception as e:
            print(f"Error processing JSON file: {str(e)}")
            import traceback
            traceback.print_exc()
            raise
    
    return lower_obj_path, upper_obj_path

def convert_ply_to_obj(ply_path, obj_path):
    """
    Convert PLY to OBJ format while preserving vertex order
    """
    vertices, rgb, faces = read_ply_file(ply_path)
    vertex_count = len(vertices)
    
    with open(obj_path, 'w') as f:
        # Write vertices with RGB values
        for v, c in zip(vertices, rgb):
            f.write(f"v {v[0]} {v[1]} {v[2]} {c[0]} {c[1]} {c[2]}\n")
        
        # Write faces (PLY uses 0-based indices, OBJ uses 1-based)
        for face in faces:
            f.write(f"f {face[0]} {face[1]} {face[2]}\n")
    
    return obj_path, vertex_count


In [ ]:
# upper_ply_path = "upper_jaw_2.ply"
lower_ply_path = "lower_jaw_2.ply"
labels_json_path = "lower_jaw_2.json"

In [4]:
a,b = process_jaw_ply_files(lower_ply_path, upper_ply_path, labels_json_path=None)

Read PLY file: 72958 vertices, 144582 faces
Read PLY file: 60704 vertices, 120868 faces


In [5]:
a

'lower_jaw_2.obj'

In [6]:
b

'upper_jaw_2.obj'